# Fake Lambda Test

**Problem**: Normally, when using quantifiers during the semantics construction, we simply introduce new variables from scratch.
For example *every integer is even* becomes $\forall x.\text{even}(x)$.
However, in mathematical text variables can be given explicit names:
*for every integer n, n is even*.
The *n* nodes in the parse tree are constants, which means that we need some special trick to convert them into variables.

**Solution**: We create a "fake lambda", i.e. a function that has the signature of lambdas and binds constants.
In the semantics construction process, the fake lambdas are replaced by real lambdas and constants get converted to variables.

**Purpose of this notebook**: Test whether the fake lambda really solves the problem as we expected.


In [1]:
subdir fakelambda

Changed to subdirectory fakelambda

Successfully reloaded GF

Here is how the theory looks like that creates fake lambdas: (the actual theory is in `COMMA/GLF`)

In [2]:
theory fakeLambda : ur:?PLF =
    fake_lambda : {a:type,b:type} a ⟶ b ⟶ (a ⟶ b) ❘ # λ 3 . 4 ❙
❚

Created theory fakeLambda

In [3]:
abstract Grammar = {
    cat
        CN; NP; VP; V2; S; Var;
    fun
        for_every : CN -> Var -> S -> S;
        exists_sth : CN -> Var -> S -> S;
        npvp : NP -> VP -> S;
        v2np : V2 -> NP -> VP;
        
        some_cn : CN -> Var -> NP;
        every_cn : CN -> Var -> NP;
        
        x, y : Var;
        novar : Var;
        integer : CN;
        divide : V2;
}

Defined Grammar

In [4]:
concrete GrammarEng of Grammar = {
  lincat
    CN = Str ;
    NP = Str ;
    VP = Str ;
    V2 = Str ;
    S = Str ;
    Var = Str ;

  lin
    for_every cn var s = "for" ++ "every" ++ cn ++ var;
    exists_sth cn var s = "there" ++ "is" ++ "an" ++ cn ++ var;
    npvp np vp = np ++ vp;
    v2np v2 np = v2 ++ np;
    some_cn cn var = "some" ++ cn ++ var ;
    every_cn cn var = "every" ++ cn ++ var ;
    x = "X" ;
    y = "Y" ;
    novar = "" ;
    integer = "integer" ;
    divide = "divides" ;
}

In [5]:
gr -number=5 | l

for every integer X

for every integer X

every integer X divides some integer

every integer X divides every integer Y

for every integer Y

In [6]:
theory logic : ur:?LF =
    prop : type ❘ # o ❙
    not : o ⟶ o ❘ # ¬ 1 prec 80 ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 prec 70 ❙
    or : o ⟶ o ⟶ o ❘ # 1 ∨ 2 prec 60 ❙
    impl : o ⟶ o ⟶ o ❘ # 1 ⇒ 2 prec 50 ❙
    equiv : o ⟶ o ⟶ o ❘ # 1 ⇔ 2 prec 40 ❙
    
    ind : type ❘ # ι ❙
    forall : (ι ⟶ o) ⟶ o ❘ # ∀ 1 ❙
    exists : (ι ⟶ o) ⟶ o ❘ # ∃ 1 ❙
    
    // fake_lambda : ι ⟶ o ⟶ (ι ⟶ o) ❘ # fλ 1 . 2 ❙
    include ☞http://mathhub.info/COMMA/GLF?fakeLambda ❙
❚

Created theory logic

In [7]:
theory dt : ?logic =
    divide : ι ⟶ ι ⟶ o ❙
    integer : ι ⟶ o ❙
    xvar : ι ❙
    yvar : ι ❙
    novar : ι ❙
❚

Created theory dt

In [8]:
view GrammarSemantics : http://mathhub.info/comma/jupyter/fakelambda/Grammar.gf?Grammar -> ?dt =
    CN = ι ⟶ o ❙
    NP = (ι ⟶ o) ⟶ o ❙
    VP = ι ⟶ o ❙
    V2 = ι ⟶ ι ⟶ o ❙
    S = o ❙
    Var = ι ❙

    for_every = [cn, var, s] ∀ λ var . (cn var) ⇒ s ❙
    exists_sth = [cn, var, s] ∃ λ var . (cn var) ∧ s ❙
    npvp = [np,vp] np vp ❙
    v2np = [v2,np] [x] np (v2 x) ❙
    some_cn = [cn,var] [p] ∃ λ var . (cn var) ∧ (p var) ❙
    every_cn = [cn,var] [p] ∀ λ var . (cn var) ⇒ (p var) ❙
    x = xvar ❙
    y = yvar ❙
    novar = novar ❙
    integer = integer ❙
    divide = divide ❙
❚

Created view GrammarSemantics

In [9]:
p "every integer X divides some integer"

npvp (every_cn integer x) (v2np divide (some_cn integer novar))

In [10]:
p "every integer X divides some integer" | construct

∀[xvar:ι](integer xvar)⇒∃[novar:ι](integer novar)∧(divide xvar novar)